<a href="https://colab.research.google.com/github/arkrom43/rocket/blob/main/SecurityDistanceCalculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
from typing import Tuple, Dict
from enum import Enum

class 推進剤種類(str, Enum):
    固体推進剤 = "固体推進剤"
    火工品 = "火工品"
    ヒドラジン類_NTO = "ヒドラジン類_NTO"
    LOX_LH2 = "LOX_LH2"
    アルコール系_LOX = "アルコール系_LOX"

def get_tnt_conversion_rates(propellant_type: str, wp: float) -> Tuple[float, float]:
    """TNT換算率(Tw, Tiw)を返す"""
    conversions = {
        推進剤種類.固体推進剤: (0.05, 0.05),
        推進剤種類.火工品: (1.0, 1.0),
        推進剤種類.ヒドラジン類_NTO: (0.1, 0.1),
        推進剤種類.LOX_LH2: (6.7/wp**(1/3), 7.8/wp**(1/3)),
        推進剤種類.アルコール系_LOX: (0.2, 0.2)
    }
    return conversions[propellant_type]

def calculate_z_value(tiw: float, wp: float, r: float) -> float:
    """Zを計算"""
    return r / (tiw * wp)**(1/3)

def calculate_impulse(tiw: float, wp: float, z: float) -> float:
    """インパルスIを計算"""
    return (tiw * wp)**(1/3) * 367 * z**(-1.08 + 0.0072 * math.log(z))

def calculate_delta_p(i: float) -> float:
    """ΔPをインパルスから計算"""
    if i <= 140:
        return 1.379
    elif 140 < i < 400:
        return 1.379 * (i/140)**0.24
    else:
        return 1.073

def calculate_blast_distance(wp: float, propellant_type: str) -> float:
    """爆風に対する保安距離R計算"""
    tw, tiw = get_tnt_conversion_rates(propellant_type, wp)

    r = 100.0
    z = calculate_z_value(tiw, wp, r)
    i = calculate_impulse(tiw, wp, z)
    delta_p = calculate_delta_p(i)
    r_new = (74/delta_p**(1/1.41)) * (tw * wp)**(1/3)

    while abs(r - r_new) > 0.1:
        r = r_new
        z = calculate_z_value(tiw, wp, r)
        i = calculate_impulse(tiw, wp, z)
        delta_p = calculate_delta_p(i)
        r_new = (74/delta_p**(1/1.41)) * (tw * wp)**(1/3)

    return r_new

def calculate_fragment_distance(wp: float, has_solid: bool) -> float:
    """飛散物に対する保安距離D計算"""
    if has_solid:
        return 117 * wp**0.31
    return 59 * wp**0.21  # 指数を0.31から0.21に修正

def calculate_fireball_distance_solid(wp: float, tw: float) -> float:
    """固体推進剤のファイアボール保安距離F計算"""
    tb = 0.258 * (tw * wp)**0.349
    ib = (550000/tb)**(1/1.15)
    f1 = math.sqrt((2.69e5 * (tw * wp)**0.78)/ib)

    f2 = math.sqrt(2.14e3 * (tw * wp)**0.78)

    return max(f1, f2)

def calculate_fireball_distance_liquid(wp: float) -> float:
    """液体推進剤のファイアボール保安距離F計算"""
    # パターンA
    tl = 1.82 * wp**(1/6)
    il = (550000/tl)**(1/1.15)
    fa = math.sqrt((8.58e6 * wp**(2/3))/il)

    # パターンB
    il_fixed = 12560
    fb = math.sqrt((8.58e6 * wp**(2/3))/il_fixed)

    return max(fa, fb)

def solve_mixed_fireball_equations(tb: float, tl: float) -> Tuple[float, float]:
    """混合推進剤の連立方程式を解く"""
    def equation_system(x):
        ib, il = x
        if tl >= tb:
            eq1 = tb * (il + ib)**1.15 + (tl - tb) * il**1.15 - 550000
        else:
            eq1 = tl * (il + ib)**1.15 + (tb - tl) * ib**1.15 - 550000
        eq2 = ib + il - 12560
        return [eq1, eq2]

    ib = il = 6280
    for _ in range(100):
        eq1, eq2 = equation_system([ib, il])
        if abs(eq1) < 1 and abs(eq2) < 1:
            break
        ib = il = ib - eq1/1000

    return ib, il

def calculate_fireball_distance_mixed(wp_solid: float, wp_liquid: float, tw: float) -> float:
    """混合推進剤のファイアボール保安距離F計算"""
    tb = 0.258 * (tw * wp_solid)**0.349
    tl = 1.82 * wp_liquid**(1/6)

    ib, il = solve_mixed_fireball_equations(tb, tl)

    if tl >= tb:
        f1 = math.sqrt((2.69e5 * (tw * wp_solid)**0.78)/ib)
    else:
        f1 = math.sqrt((8.58e6 * wp_liquid**(2/3))/il)

    f2 = max(
        math.sqrt(2.14e3 * (tw * wp_solid)**0.78),
        math.sqrt(683 * wp_liquid**(2/3))
    )

    return max(f1, f2)

def 保安距離計算(推進剤質量: Dict[str, float]) -> Dict[str, float]:
    """
    推進剤の質量から保安距離を計算
    """
    total_mass = sum(推進剤質量.values())
    has_solid = (推進剤種類.固体推進剤 in 推進剤質量 or
                推進剤種類.火工品 in 推進剤質量)

    # 爆風距離計算
    r_distances = [
        calculate_blast_distance(mass, prop_type)
        for prop_type, mass in 推進剤質量.items()
    ]

    # 飛散物距離計算
    d_distance = calculate_fragment_distance(total_mass, has_solid)

    # ファイアボール距離計算
    if has_solid and len(推進剤質量) > 1:
        solid_mass = sum(mass for type_, mass in 推進剤質量.items()
                        if type_ in [推進剤種類.固体推進剤, 推進剤種類.火工品])
        liquid_mass = total_mass - solid_mass
        tw, _ = get_tnt_conversion_rates(推進剤種類.固体推進剤, solid_mass)
        f_distance = calculate_fireball_distance_mixed(solid_mass, liquid_mass, tw)
    elif has_solid:
        tw, _ = get_tnt_conversion_rates(推進剤種類.固体推進剤, total_mass)
        f_distance = calculate_fireball_distance_solid(total_mass, tw)
    else:
        f_distance = calculate_fireball_distance_liquid(total_mass)

    return {
        "爆風距離": max(r_distances),
        "飛散物距離": d_distance,
        "ファイアボール距離": f_distance,
        "最終保安距離": max(max(r_distances), d_distance, f_distance)
    }

#ここの部分を書き換えてください
if __name__ == "__main__":
    推進剤 = {
        推進剤種類.アルコール系_LOX: 0.5,  # ケロシン/LOX 34500kg
        推進剤種類.LOX_LH2: 1.4
    }

    結果 = 保安距離計算(推進剤)
    print("保安距離計算結果 (メートル):")
    for key, value in 結果.items():
        print(f"{key}: {value:.2f}")


保安距離計算結果 (メートル):
爆風距離: 119.70
飛散物距離: 67.51
ファイアボール距離: 32.37
最終保安距離: 119.70
